### Audioset Processing

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import os

2023-06-03 13:55:02.123724: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-03 13:55:02.163946: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-03 13:55:02.164564: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-03 13:55:03.062607: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
directory = "audioset_v1_embeddings/eval/"

dataset = []
for file_name in os.listdir(directory):
    if file_name.endswith(".tfrecord"):
        dataset.append(os.path.join(directory, file_name))

In [4]:
tf.compat.v1.enable_eager_execution()

In [5]:
raw_dataset = tf.data.TFRecordDataset(dataset)

In [6]:
# Create a list of distinguishable sounds
class_labels = pd.read_csv("class_labels_indices.csv")
labels = class_labels["display_name"].tolist()

# TODO: Include music genre labels that do not contain the word 'music'
# Create a list for music-genre-specific labels
music_class = class_labels[class_labels["display_name"].str.contains("Music", case=False)]
music_labels = music_class["index"].tolist()

In [10]:
audios = []
counter = 0
NUM_SECONDS = 10

for raw_record in raw_dataset:
    example = tf.train.SequenceExample()
    example.ParseFromString(raw_record.numpy())

    # Audio Meta Data
    audio_labels = example.context.feature["labels"].int64_list.value
    start_time = example.context.feature["start_time_seconds"].float_list.value
    end_time = example.context.feature["end_time_seconds"].float_list.value
    video_id = example.context.feature["video_id"].bytes_list.value

    if not (set(music_labels) & set(music_labels)):
        continue

    # Audio Feature
    feature_list = example.feature_lists.feature_list["audio_embedding"].feature
    final_features = [list(feature.bytes_list.value[0]) for feature in feature_list]
    audio_embedding = [item for sublist in final_features[:NUM_SECONDS] for item in sublist]

    if len(final_features) > NUM_SECONDS:
        continue

    audio = {
        "label": audio_labels,
        "video_id": video_id,
        "start_time": start_time,
        "end_time": end_time,
        "data": audio_embedding
    }

    audios.append(audio)
    counter += 1
    if (counter % 100 == 0):
        print(f"Processing {counter}th file...")

Processing 100th file...
Processing 200th file...
Processing 300th file...
Processing 400th file...
Processing 500th file...
Processing 600th file...
Processing 700th file...
Processing 800th file...
Processing 900th file...
Processing 1000th file...
Processing 1100th file...
Processing 1200th file...
Processing 1300th file...
Processing 1400th file...
Processing 1500th file...
Processing 1600th file...
Processing 1700th file...
Processing 1800th file...
Processing 1900th file...
Processing 2000th file...
Processing 2100th file...
Processing 2200th file...
Processing 2300th file...
Processing 2400th file...
Processing 2500th file...
Processing 2600th file...
Processing 2700th file...
Processing 2800th file...
Processing 2900th file...
Processing 3000th file...
Processing 3100th file...
Processing 3200th file...
Processing 3300th file...
Processing 3400th file...
Processing 3500th file...
Processing 3600th file...
Processing 3700th file...
Processing 3800th file...
Processing 3900th fil

In [11]:
# Write results to a JSON file
with open("music_set.json", "w") as file:
    str_audio = repr(audios)
    json.dump(str_audio, file)

In [13]:
[audio["data"][:10] + audio["data"][-10:] for audio in audios[:3]]

[[46,
  178,
  112,
  12,
  255,
  110,
  224,
  138,
  183,
  151,
  88,
  54,
  241,
  158,
  0,
  156,
  255,
  0,
  177,
  37],
 [144,
  97,
  61,
  98,
  180,
  3,
  113,
  115,
  97,
  130,
  167,
  106,
  3,
  55,
  96,
  199,
  189,
  0,
  184,
  255],
 [73,
  232,
  131,
  57,
  162,
  124,
  124,
  111,
  255,
  126,
  0,
  0,
  170,
  106,
  107,
  0,
  255,
  102,
  142,
  61]]